<a href="https://colab.research.google.com/github/pakka79/pakka79/blob/main/CnG_Stock_crawling_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libary===================================================#
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pykrx import stock
import datetime as dt
import pytz
import pandas as pd
import FinanceDataReader as fdr
import tqdm
#Libary===================================================#

In [ ]:
#DATE===================================================#

KoreaTime = pytz.timezone('Asia/Seoul')
From_Today=1000
today = dt.datetime.now()
previouday=today-dt.timedelta(days=From_Today)
Today=today.strftime("%Y%m%d")
Previuosday=previouday.strftime("%Y%m%d")
print(Today)

From_Today_Dart=0
# TodayDart="2022-08-25"
# PreviuosdayDart="2022-08-24"
TodayDart=today.strftime("%Y-%m-%d")
previoudaydart=today-dt.timedelta(days=From_Today_Dart)
PreviuosdayDart=previoudaydart.strftime("%Y-%m-%d")
print(TodayDart)

#DATE===================================================#

In [ ]:
#KRX_Chart===================================================#
KRX_Chart=pd.read_csv("/Volumes/Kraphs_Coding/Python_Project/[Daily_Chart]/[Daily_Chart]_{}_KRX_Ticker.csv".format(Today))
KRX_Chart=KRX_Chart[['KRX_TICKER_LIST', 'Market', 'Name', 'Sector','Industry']]
KRX_TICKER_LIST=[]

for k in KRX_Chart['KRX_TICKER_LIST']:
    KRX_TICKER_LIST.append(str(k).zfill(6))

KRX_Chart['KRX_TICKER_LIST']=KRX_TICKER_LIST
KRX_Chart
#KRX_Chart===================================================#

In [ ]:
#Type===================================================#
D="D" #연결
B="B" #개별
Type=D
#Type===================================================#

In [ ]:
#연결#
S=0

for i in KRX_Chart['KRX_TICKER_LIST']:
    try:
        
        hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=" + "A" + i + "&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701"
        a = requests.get(url=url, headers=hdr)
        b = BeautifulSoup(a.content, 'html.parser')
        c = b.find("div", {"id": "highlight_"+Type+"_Q"}) #B_Q:개별분기,D_Q:연결분기
        df = pd.read_html(str(c))[0]
        df = df.transpose()
        
        df["No"]=range(0,9)
        df["KRX_TICKER_LIST"]=i
        df["Current"]='=GOOGLEFINANCE("{}","price")'.format(i)
        df['PER_당기순이익_(Live)'] = '=IFERROR(E' + str(6 + S * 9) + "/" + 'AH' + str(6 + S * 9) + ")"
        df['PER_영업순이익_(Live)'] = '=IFERROR(E' + str(6 + S * 9) + "/" + '(M' + str(6 + S * 9) + "*100000000" + '/' + 'AO' + str(6 + S * 9) + "/1000"+ ')' + ')'
        S = S + 1
        df["Corp"]=stock.get_market_ticker_name(i)
        df.rename(columns={0:"매출액",1:"영업이익",2:"영업이익(발표기준)",3:"당기순이익",4:"지배주주순이익",5:"비지배주주순이익",6:"자산총계",7:"부채총계",8:"자본총계",9:"지배주주지분",10:"비지배주주지분",11:"자본금",12:"부채비율",13:"유보율",14:"영업이익률",15:"지배주주순이익률",16:"ROA",17:"ROE",18:"EPS(원)",19:"BPS(원)",20:"DPS(원)",21:"PER",22:"PBR",23:"발행주식수",24:"배당수익률"}, inplace=True)
        df=df[1:9]
        print(df.columns)
        
#매출액===================================================#

        df['EPS_3AVE'] = round(df['EPS(원)'].rolling(3).mean(), 2)
        df['ROE_3AVE'] = round(df['ROE'].rolling(3).mean(), 2)
        df['ROA_3AVE'] = round(df['ROA'].rolling(3).mean(), 2)

        df['매출액_Consensus']=round(df['매출액'].rolling(4).sum())
        df['매출액(Before)'] = df['매출액'].shift(1)
        df['매출액증가분'] = df['매출액'] - df['매출액(Before)']
        
        매출액_Filter = []
        for i in range(0, 8):
            if df['매출액'][1] - df['매출액'][0] > 0 \
                    and df['매출액'][2] - df['매출액'][1] > 0 \
                    and df['매출액'][3] - df['매출액'][2] > 0 \
                    and df['매출액'][4] - df['매출액'][3] > 0:
                매출액_Filter.append("OK")
                # print(campanyname,"OK")
            else:
                매출액_Filter.append("--")
                # print(campanyname,"--")
        df["매출액_Filter"] = 매출액_Filter

#매출액===================================================#

#영업이익===================================================#
        df['영업이익_Consensus'] = round(df['영업이익'].rolling(4).sum())
        df['영업이익(Before)'] = df['영업이익'].shift(1)
        df['영업이익증가분'] = df['영업이익'] - df['영업이익(Before)']
        영업이익_Filter = []
        for i in range(0, 8):
            if df['영업이익'][1] - df['영업이익'][0] > 0 \
                    and df['영업이익'][2] - df['영업이익'][1] > 0 \
                    and df['영업이익'][3] - df['영업이익'][2] > 0 \
                    and df['영업이익'][4] - df['영업이익'][3] > 0:
                영업이익_Filter.append("OK")
                # print(campanyname,"OK")
            else:
                영업이익_Filter.append("--")
                # print(campanyname,"--")
        df["영업이익_Filter"] = 영업이익_Filter
#영업이익===================================================#

#EPS(원)===================================================#
        
        df['EPS_Consensus'] = round(df['EPS(원)'].rolling(4).sum())
        df['EPS(Before)'] = df['EPS(원)'].shift(1)
        df['EPS증가분'] = df['EPS(원)'] - df['EPS(Before)']

        EPS_Filter = []
        for i in range(0, 8):
            if df['EPS(원)'][1] - df['EPS(원)'][0] > 0 \
                    and df['EPS(원)'][2] - df['EPS(원)'][1] > 0 \
                    and df['EPS(원)'][3] - df['EPS(원)'][2] > 0 \
                    and df['EPS(원)'][4] - df['EPS(원)'][3] > 0:
                EPS_Filter.append("OK")
                # print(campanyname,"OK")

            else:
                EPS_Filter.append("--")
                # print(campanyname,"--")
        df["EPS_Filter"] = EPS_Filter
#EPS(원)===================================================#

        df=pd.merge(KRX_Chart,df,on="KRX_TICKER_LIST")
        df=df[['KRX_TICKER_LIST','Market','Name','Current','Sector','Industry','매출액','매출액_Consensus','매출액_Filter','영업이익','영업이익(발표기준)','영업이익_Consensus','영업이익_Filter','당기순이익','지배주주순이익','비지배주주순이익','자산총계','부채총계','자본총계','지배주주지분','비지배주주지분','자본금','부채비율','유보율','영업이익률','지배주주순이익률','ROA','ROE_3AVE','ROE','ROE_3AVE','EPS(원)','EPS_3AVE','EPS_Consensus','EPS_Filter','BPS(원)','DPS(원)','PER_당기순이익_(Live)','PER_영업순이익_(Live)','PBR','발행주식수','배당수익률']]
        print(df)
        df.to_csv("{}_CnG_Q{}_Finance_Main.csv".format(Today,Type),mode="a",header="None")

    except:
        pass


In [ ]:
#별도#
S=0

for i in KRX_Chart['KRX_TICKER_LIST']:
    try:
        
        hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=" + "A" + i + "&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701"
        a = requests.get(url=url, headers=hdr)
        b = BeautifulSoup(a.content, 'html.parser')
        c = b.find("div", {"id": "highlight_"+Type+"_Q"}) #B_Q:개별분기,D_Q:연결분기
        df = pd.read_html(str(c))[0]
        df = df.transpose()
        
        df["No"]=range(0,9)
        df["KRX_TICKER_LIST"]=i
        df["Current"]='=GOOGLEFINANCE("{}","price")'.format(i)
        df['PER_당기순이익_(Live)'] = '=IFERROR(E' + str(6 + S * 9) + "/" + 'AH' + str(6 + S * 9) + ")"
        df['PER_영업순이익_(Live)'] = '=IFERROR(E' + str(6 + S * 9) + "/" + '(M' + str(6 + S * 9) + "*100000000" + '/' + 'AO' + str(6 + S * 9) + "/1000"+ ')' + ')'
        S = S + 1
        df["Corp"]=stock.get_market_ticker_name(i)
        df.rename(columns={0:"매출액",1:"영업이익",2:"영업이익(발표기준)",3:"당기순이익",4:"자산총계",5:"부채총계",6:"자본총계",7:"자본금",8:"부채비율",9:"유보율",10:"영업이익률",11:"순이익률",12:"ROA",13:"ROE",14:"EPS(원)",15:"BPS(원)",16:"DPS(원)",17:"PER",18:"PBS",19:"발행주식수",20:"배당수익률"}, inplace=True)
        df=df[1:9]
        
#매출액===================================================#

        df['EPS_3AVE'] = round(df['EPS(원)'].rolling(3).mean(), 2)
        df['ROE_3AVE'] = round(df['ROE'].rolling(3).mean(), 2)
        df['ROA_3AVE'] = round(df['ROA'].rolling(3).mean(), 2)

        df['매출액_Consensus']=round(df['매출액'].rolling(4).sum())
        df['매출액(Before)'] = df['매출액'].shift(1)
        df['매출액증가분'] = df['매출액'] - df['매출액(Before)']
        
        매출액_Filter = []
        for i in range(0, 8):
            if df['매출액'][1] - df['매출액'][0] > 0 \
                    and df['매출액'][2] - df['매출액'][1] > 0 \
                    and df['매출액'][3] - df['매출액'][2] > 0 \
                    and df['매출액'][4] - df['매출액'][3] > 0:
                매출액_Filter.append("OK")
                # print(campanyname,"OK")
            else:
                매출액_Filter.append("--")
                # print(campanyname,"--")
        df["매출액_Filter"] = 매출액_Filter

#매출액===================================================#

#영업이익===================================================#
        df['영업이익_Consensus'] = round(df['영업이익'].rolling(4).sum())
        df['영업이익(Before)'] = df['영업이익'].shift(1)
        df['영업이익증가분'] = df['영업이익'] - df['영업이익(Before)']
        영업이익_Filter = []
        for i in range(0, 8):
            if df['영업이익'][1] - df['영업이익'][0] > 0 \
                    and df['영업이익'][2] - df['영업이익'][1] > 0 \
                    and df['영업이익'][3] - df['영업이익'][2] > 0 \
                    and df['영업이익'][4] - df['영업이익'][3] > 0:
                영업이익_Filter.append("OK")
                # print(campanyname,"OK")
            else:
                영업이익_Filter.append("--")
                # print(campanyname,"--")
        df["영업이익_Filter"] = 영업이익_Filter
#영업이익===================================================#

#EPS(원)===================================================#
        
        df['EPS_Consensus'] = round(df['EPS(원)'].rolling(4).sum())
        df['EPS(Before)'] = df['EPS(원)'].shift(1)
        df['EPS증가분'] = df['EPS(원)'] - df['EPS(Before)']

        EPS_Filter = []
        for i in range(0, 8):
            if df['EPS(원)'][1] - df['EPS(원)'][0] > 0 \
                    and df['EPS(원)'][2] - df['EPS(원)'][1] > 0 \
                    and df['EPS(원)'][3] - df['EPS(원)'][2] > 0 \
                    and df['EPS(원)'][4] - df['EPS(원)'][3] > 0:
                EPS_Filter.append("OK")
                # print(campanyname,"OK")

            else:
                EPS_Filter.append("--")
                # print(campanyname,"--")
        df["EPS_Filter"] = EPS_Filter
#EPS(원)===================================================#

        df=pd.merge(KRX_Chart,df,on="KRX_TICKER_LIST")
        df=df[['KRX_TICKER_LIST','Market','Name','Current','Sector','Industry','매출액','매출액_Consensus','매출액_Filter','영업이익','영업이익(발표기준)','영업이익_Consensus','영업이익_Filter','당기순이익','자산총계','부채총계','자본총계','자본금','부채비율','유보율','영업이익률',"순이익률",'ROA','ROE_3AVE','ROE','ROE_3AVE','EPS(원)','EPS_3AVE','EPS_Consensus','EPS_Filter','BPS(원)','DPS(원)','PER_당기순이익_(Live)','PER_영업순이익_(Live)','PBS','발행주식수','배당수익률']]
        print(df)
        df.to_csv("{}_CnG_Q{}_Finance_Main.csv".format(Today,Type),mode="a",header="None")

    except:
        pass
